<div align='left' style="width:29%;overflow:hidden;">
<a href='http://inria.fr'>
<img src='https://github.com/lmarti/jupyter_custom/raw/master/imgs/inr_logo_rouge.png' alt='Inria logo' title='Inria logo'/>
</a>
</div>

# RISOTTO

> A work in progress GUI for RISOTTO

In [ ]:
!pip install --disable-pip-version-check -q -r requirements.txt

In [ ]:
from collections import defaultdict

from ipywidgets import GridspecLayout, Layout, Label, Box, HBox, VBox, HTML, widgets, Button
from IPython.display import display

from risotto.artifacts import load_papers_artifact, load_papers_topics_artifacts, load_topics_artifacts

In [ ]:
papers_artifact = load_papers_artifact()
f"https://doi.org/{papers_artifact.iloc[0]["doi"]}"

'10.1186/1471-2334-1-6'

In [ ]:
"""
This notebook implements RISOTTO's GUI.
The GUI is structured as the following tree:

> VBox
    > FilterAndSearchView
    > Box
        > PaperSetView
            > VBox
                > PaperView_1
                > PaperView_2
                > ...
                > PaperView_n
                > PaperSetView.nav_widgets
"""


# The artifacts built in the cooking stage are loaded
papers_artifact = load_papers_artifact().fillna("N/A")
papers_topics_artifact = load_papers_topics_artifacts()
topics_artifact = load_topics_artifacts()


class FilterAndSearchView:
    """
    This view is responsible for rendering the filter and search widgets.
    """
    
    # Constant defining how many relevant tokens are displayed in the options
    TOKENS_PER_TOPIC = 5
    
    def __init__(self, topics_artifact, on_filter_handler):
        """
        Args:
            - topics_artifact: a Pandas DataFrame with the tokens pesudocounts
                of the each topic and subtopic.
            - on_filter_handler: a function that's called when the 'Filter'
                button is pressed. It receives as parameters the selected
                topic and subtopic identifier, and the search text field value.
        """
        self._topics_artifact = topics_artifact
        self._on_filter_handler = on_filter_handler
        self._topics, self._subtopics = self._get_topics(topics_artifact)
    
    @classmethod
    def _get_topics(cls, topics_artifact):
        """
        Args:
            - topics_artifact: a Pandas DataFrame with the tokens pesudocounts
                of the each topic and subtopic.
        Returns:
            - topics: list of tuples. Each tuple's first and second elements are
                the topic's readable name and identifier, respectively.
            - subtopics: dictionary with topic ids as key and list of tuples as
                values. Each tuple's first and second elements are
                the subtopic's readable name and identifier, respectively.
        """
        topics = []
        subtopics = defaultdict(list)
        for col_name in topics_artifact.columns:
            if "-" not in col_name:
                topics.append((cls._get_dropdown_name(topics_artifact[col_name]), col_name))
            else:
                topic_id = col_name.split("-")[0]
                subtopics[topic_id].append((cls._get_dropdown_name(topics_artifact[col_name]), col_name))
        return topics, subtopics
    
    @classmethod
    def _get_dropdown_name(cls, series):
        """
        Args:
            - series: a Pandas Series with the tokens pseudocounts of a topic
                or subtopic
        Returns:
            - str: a readable name of the topic or subtopic
        """
        prefix = f"#{series.name}"
        most_relevant = ", ".join(map(str, series.sort_values(ascending=False).index[:cls.TOKENS_PER_TOPIC]))
        return f"{prefix} ({most_relevant})"
        
    def to_widget(self):
        topics_dropdown = widgets.Dropdown(
            options=[("Todos", None)] + self._topics,
            description="Tópico:",
        )
        subtopics_dropdown = widgets.Dropdown(
            options=[("Todos", None)] + self._subtopics[topics_dropdown.value],
            description="Subtópico:",
        )
        search_textbox = widgets.Text(
            description="Búsqueda:",
        )
        search_button = widgets.Button(
            description="Filtrar",
        )
        dropdowns_box = HBox([
            topics_dropdown,
            subtopics_dropdown,
        ])
        search_box = HBox([
            search_textbox,
            search_button,
        ])

        # Event handlers
        def handle_topic_change(event):
            if event["type"] == "change" and event["name"] == "value":
                subtopics_dropdown.options = [("Todos", None)] + self._subtopics[event["new"]]

        def handle_subtopic_change(event):
            if event["type"] == "change" and event["name"] == "value":
                pass
        
        def handle_search_button_click(_):
            self._on_filter_handler(
                topic_id=topics_dropdown.value,
                subtopic_id=subtopics_dropdown.value,
                text=search_textbox.value
            )
            
        topics_dropdown.observe(handle_topic_change)
        subtopics_dropdown.observe(handle_subtopic_change)
        search_button.on_click(handle_search_button_click)
        
        filter_and_search_box = VBox([
            dropdowns_box,
            search_box,
        ])
        return filter_and_search_box


class PaperView:
    """
    This view is responsible for rendering a paper information.
    """
    
    def __init__(self, row):
        """
        Args:
            - row: a Pandas Series with the paper's data.
        """
        self._row = row
        
    def to_widget(self):
        grid = GridspecLayout(n_rows=7, n_columns=2, justify_content='space-around', align_items='flex-start')

        grid[0,0] = HTML('<span style="color: red;">Title</span>:', layout=Layout(height='auto', width='20%'))
        grid[0,1] = Label(self._row.title)
        grid[1,0] = Label('Authors:')
        grid[1,1] = Label(self._row.authors)
        grid[2,0] = Label('Date:', layout=Layout(height='auto', width='20%'))
        grid[2,1] = Label(self._row.publish_time)
        grid[3,0] = Label('Relevance:', layout=Layout(height='auto', width='20%'))
        grid[3,1] = Label(str(self._row.pagerank))
        grid[4,0] = Label('URL:',layout=Layout(height='auto', width='20%'))
        grid[4,1] = HTML(f'<a href="https://doi.org/{self._row.doi}">https://doi.org/{self._row.doi}</a>')
        grid[5,0] = Label('DOI:',layout=Layout(height='auto', width='20%'))
        grid[5,1] = Label(f'{self._row.doi}')
        grid[6,0] = Label('Abstract:', layout=Layout(height='auto', width='20%'))
        grid[6,1] = Label(str(self._row.abstract))

        return grid

    
class PaperSetView:
    """
    This view is responsible for rendering a set of papers.
    Also, the view implements pagination features.
    """
    
    def __init__(self, papers, papers_topics, page, items_per_page, parent):
        """
        Args:
            - papers: a Pandas DataFrame with the data of the papers,
                including the `cord_uid` identifier and the PageRank scores.
            - papers_topics: a Pandas DataFrame with the association between
                papers, topics, and subtopics
            - topics_artifact: a Pandas DataFrame with the tokens pesudocounts
                of the each topic and subtopic.
            - page: first page to render.
            - items_per_page: how many items to render in each page.
            - parent: the parent widget. It's required for rerendering the
                component on updates.
        """
        self._papers = papers.join(papers_topics).sort_values(by="pagerank", ascending=False)
        self._page = page
        self._items_per_page = items_per_page
        self._parent = parent
        
        self._topic_id = None
        self._subtopic_id = None
        self._query_text = ""
        
        self._render()
    
    def _decrement_page(self, _):
        self._page += -1
        self._render()
    
    def _increment_page(self, _):
        self._page += 1
        self._render()
    
    def _render(self):
        """
        This method performs the view rendering, filtering the papers
        by the selected topic and subtopic, and then by the text query.
        It's able to rerender the component on updates modifying the
        parent's children attribute.
        """
        papers = self._papers
        # Topic filtering
        if self._topic_id is not None:
            mask = papers["topic"] == self._topic_id
            if self._subtopic_id is not None:
                mask = mask & (papers["subtopic"] == self._subtopic_id)
            papers = papers[mask]
        # Query filtering
        # mask = (papers["title"] + papers["abstract"]).str.lower().str.contains(self._query_text)
        
        self._papers_views = self.build_papers_views(papers, self._page, self._items_per_page)
        self._parent.children = [self.to_widget(papers)]
        
    @staticmethod
    def build_papers_views(papers, page, items_per_page):
        """
        Args:
            - papers: the filtered Pandas DataFrame with the papers information.
            - page: the page to render.
            - items_per_page: how many items to render in each page.
        Returns:
            - list with the current page PaperViews.
        """
        start_idx, end_idx = page * items_per_page, (page + 1) * items_per_page
        page_subset = papers.iloc[start_idx:end_idx]
        return [PaperView(row) for _, row in page_subset.iterrows()]
    
    @staticmethod
    def get_nav_widgets(page, items_per_page, num_items, decrement_handler, increment_handler):
        """
        Args:
            - page: current page
            - items_per_page: how many items to display each page
            - num_items: total number of items
            - decrement_handler: function that it's called when the previous page
                button is clicked
            - increment_handler: function that it's called when the nextpage
                button is clicked
        Returns:
            - HBox with the navigation widgets
        """
        last_page = num_items // items_per_page
        widgets = []
        if page > 0:
            button = Button(description="Página anterior")
            button.on_click(decrement_handler)
            widgets.append(button)
        widgets.append(HTML(f"Página {page + 1} de {last_page + 1}. Mostrando {items_per_page} elementos por página."))
        if page < last_page:
            button = Button(description="Página siguiente")
            button.on_click(increment_handler)
            widgets.append(button)
        return HBox(widgets)
    
    def on_filter_handler(self, topic_id, subtopic_id, text):
        """
        This metthod is invoked when the 'Filter' button is clicked.
        Args:
            - topic_id: new topic identifier
            - subtopic_id: new subtopic identifier
            - text: the search text field value
        """
        self._topic_id = int(topic_id) if topic_id is not None else None
        self._subtopic_id = int(subtopic_id.split("-")[1]) if subtopic_id is not None else None
        self._render()
    
    # def on
        
    def to_widget(self, papers):
        """
        Args:
            - papers: the filtered Pandas DataFrame with the papers information
        Returns:
            - VBox with the view widgets
        """
        papers_widgets = [view.to_widget() for view in self._papers_views]
        nav_widgets = self.get_nav_widgets(
            page=self._page,
            items_per_page=self._items_per_page,
            num_items=len(papers),
            decrement_handler=self._decrement_page,
            increment_handler=self._increment_page
        )
        box = VBox(papers_widgets + [nav_widgets])
        return box

paper_set_box = Box()
paper_set_view = PaperSetView(
    papers=papers_artifact,
    papers_topics=papers_topics_artifact,
    page=0,
    items_per_page=5,
    parent=paper_set_box
)
filter_and_search_view = FilterAndSearchView(
    topics_artifact=topics_artifact,
    on_filter_handler=paper_set_view.on_filter_handler,
)
box = VBox([
    filter_and_search_view.to_widget(),
    paper_set_box,
])
box

In [ ]:
"""
TODO:
- Integrar Whoosh para la búsqueda
- Hacer encabezado
"""

'\nTODO:\n- Integrar Whoosh para la búsqueda\n- Hacer encabezado\n'